# Build a Spam Classifier with Keras
With deep learning and AI, handling spam content has gotten easier and easier. Over time (and with the aid of direct user feedback) our spam classifier will rarely produce erroneous results.

This is the first part of a multi-part series covering how to:

- Build an AI Model (this one)
- Integrate a NoSQL Database (inference result storing)
- Deploy an AI Model into Production

### Prerequisites
- Prepare your dataset using [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb) .
- Convert your dataset into trainable vectors in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb) (Either way, this notebook will run this step for us).


### Running this notebook:
- Recommended: Use [Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb) as it offers free GPUs for training models. [Launch this notebook here]([Colab](https://colab.research.google.com/github/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/Spam_Classifier_with_Keras.ipynb))
- Fork [the AI as an API repo](https://github.com/codingforentrepreneurs/AI-as-an-API) and run `guides/spam-classifier/Spam_Classifier_with_Keras.ipynb` whenever you'd like.

This notebook is brought to in you in partnership with [DataStax](https://dtsx.io/3nRWZEG).

In [1]:
!pip install boto3
# !pip install pandas tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.4 MB/s eta 0:00:00


In [2]:
import boto3
import os
import pathlib
import pandas as pd
import pickle

In [3]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
EXPORT_DIR = pathlib.Path('/datasets/exports/')
GUIDES_DIR = pathlib.Path("/guides/spam-classifier/")
DATASET_CSV_PATH = EXPORT_DIR / 'spam-dataset.csv'
TRAINING_DATA_PATH = EXPORT_DIR / 'spam-training-data.pkl'
PART_TWO_GUIDE_PATH = GUIDES_DIR / "2 - Convert Dataset into Vectors.ipynb"

## Prepare Dataset

Creating a dataset rarely happens next to where you run the training. The below cells are a method for us to extract the needed data to perform training against.

In [5]:
!mkdir -p "$EXPORT_DIR"
!mkdir -p "$GUIDES_DIR"
!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/datasets/exports/spam-dataset.csv" -o "$DATASET_CSV_PATH"
!curl "https://raw.githubusercontent.com/codingforentrepreneurs/AI-as-an-API/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb" -o "$PART_TWO_GUIDE_PATH"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  729k  100  729k    0     0  1876k      0 --:--:-- --:--:-- --:--:-- 1881k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15408  100 15408    0     0  82955      0 --:--:-- --:--:-- --:--:-- 82838


Let's review our extracted dataset which combines two different spam datasets as outlined in [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/1%20-%20Prepare%20the%20AI%20Spam%20Classifier%20Dataset.ipynb).

In [6]:
df = pd.read_csv(DATASET_CSV_PATH)
df.head()

,label,text,source
0,ham,"Go until jurong point, crazy.. Available only ...",uci-spam-sms
1,ham,Ok lar... Joking wif u oni...,uci-spam-sms
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,uci-spam-sms
3,ham,U dun say so early hor... U c already then say...,uci-spam-sms
4,ham,"Nah I don't think he goes to usf, he lives aro...",uci-spam-sms


In [this notebook](https://github.com/codingforentrepreneurs/AI-as-an-API/blob/main/guides/spam-classifier/2%20-%20Convert%20Dataset%20into%20Vectors.ipynb) we prepare our dataset (`spam-dataset.csv`) to be fully ready for training on a model. Below is a command to run that notebook.

In [7]:
%run "$PART_TWO_GUIDE_PATH"

BASE_DIR is /
Random Index 1456
Found 9538 unique tokens.


Extract prepared training dataset results.

In [8]:
data = {}

with open(TRAINING_DATA_PATH, 'rb') as f:
    data = pickle.load(f)

> While the above code uses `pickle` to load in data, this data is actually exported via `pickle` when we execute the `%run` only a few steps ago. Since `pickle` can be unsafe to use from third-party downloaded data, we actually generate (again using `%run`) this pickle data and therefore is safe to use -- it's never downloaded.

## Transform Extracted Dataset

In [9]:
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']
labels_legend_inverted = data['labels_legend_inverted']
legend = data['legend']
max_sequence = data['max_sequence']
max_words = data['max_words']
tokenizer = data['tokenizer']

## Create our LSTM Model

In [10]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embed_dim, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 128)          35840     
                                                                 
 spatial_dropout1d (Spatial  (None, 280, 128)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 291034 (1.11 MB)
Trainable params: 291034 (1.11 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, verbose=1, epochs=epochs)

Epoch 1/5
163/163 [==============================] - 155s 919ms/step - loss: 0.3064 - accuracy: 0.8804 - val_loss: 0.1628 - val_accuracy: 0.9415
Epoch 2/5
163/163 [==============================] - 144s 884ms/step - loss: 0.1374 - accuracy: 0.9517 - val_loss: 0.1176 - val_accuracy: 0.9617
Epoch 3/5
163/163 [==============================] - 151s 927ms/step - loss: 0.1068 - accuracy: 0.9619 - val_loss: 0.1049 - val_accuracy: 0.9684
Epoch 4/5
163/163 [==============================] - 148s 907ms/step - loss: 0.0965 - accuracy: 0.9640 - val_loss: 0.1014 - val_accuracy: 0.9680
Epoch 5/5
163/163 [==============================] - 142s 867ms/step - loss: 0.0840 - accuracy: 0.9696 - val_loss: 0.0984 - val_accuracy: 0.9684


In [12]:
MODEL_EXPORT_PATH = EXPORT_DIR / 'spam-model.h5'
model.save(str(MODEL_EXPORT_PATH))

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Predict new data

In [13]:
import numpy as np

def predict(text_str, max_words=280, max_sequence = 280, tokenizer=None):
  if not tokenizer:
    return None
  sequences = tokenizer.texts_to_sequences([text_str])
  x_input = pad_sequences(sequences, maxlen=max_sequence)
  y_output = model.predict(x_input)
  top_y_index = np.argmax(y_output)
  preds = y_output[top_y_index]
  labeled_preds = [{f"{labels_legend_inverted[str(i)]}": x} for i, x in enumerate(preds)]
  return labeled_preds

In [19]:
predict("join the club for free here", max_words=max_words, max_sequence=max_sequence, tokenizer=tokenizer)

1/1 [==============================] - 0s 139ms/step


[{'ham': 0.93325955}, {'spam': 0.066740505}]

## Exporting Tokenizer & Metadata

In [21]:
import json
metadata = {
    "labels_legend_inverted": labels_legend_inverted,
    "legend": legend,
    "max_sequence": max_sequence,
    "max_words": max_words,
}

METADATA_EXPORT_PATH = EXPORT_DIR / 'spam-classifier-metadata.json'
METADATA_EXPORT_PATH.write_text(json.dumps(metadata, indent=4))

187

In [22]:
tokenizer_as_json = tokenizer.to_json()

TOKENIZER_EXPORT_PATH = EXPORT_DIR / 'spam-classifier-tokenizer.json'
TOKENIZER_EXPORT_PATH.write_text(tokenizer_as_json)

827612

We can load `tokenizer_as_json` with `tensorflow.keras.preprocessing.text.tokenizer_from_json`.

## Upload Model, Tokenizer, & Metadata to Object Storage

Object Storage options include:

- AWS S3
- Linode Object Storage
- DigitalOcean Spaces


All three of these options can use `boto3`.

In [ ]:
# AWS S3 Config
ACCESS_KEY = "<your_aws_iam_key_id>"
SECRET_KEY = "<your_aws_iam_secret_key>"

# You should not have to set this
ENDPOINT = None

# Your s3-bucket region
REGION = 'us-west-1'

BUCKET_NAME = '<your_s3_bucket_name>'

#### Linode Object Storage Config

In [23]:
ACCESS_KEY = "NS2P0D7H71RWB3IP1DXK"
SECRET_KEY = "UuB8XTIbu6lWXBhQrGteiPG71uTtv2duu7ZKtGmR"

# Object Storage Endpoint URL
ENDPOINT = "https://lstm-spam-api.us-east-1.linodeobjects.com"

# Object Storage Endpoint Region (also in your endpoint url)
REGION = 'us-east-1'

# Set this to a valid slug (without a "/" )
BUCKET_NAME = 'datasets'

#### DigitalOcean Spaces Config

In [ ]:
ACCESS_KEY = "<your_do_spaces_access_key>"
SECRET_KEY = "<your_do_spaces_secret_key>"

# Space Endpoint URL
ENDPOINT = "https://ai-cfe-1.nyc3.digitaloceanspaces.com"

# Space Region (also in your endpoint url)
REGION = 'nyc3'

# Set this to a valid slug (without a "/" )
BUCKET_NAME = 'datasets'

## Perform Upload with Boto3

In [24]:
os.environ["AWS_ACCESS_KEY_ID"] = ACCESS_KEY
os.environ["AWS_SECRET_ACCESS_KEY"] = SECRET_KEY

In [25]:
# Upload paths
MODEL_KEY_NAME = f"exports/spam-sms/{MODEL_EXPORT_PATH.name}"
TOKENIZER_KEY_NAME = f"exports/spam-sms/{TOKENIZER_EXPORT_PATH.name}"
METADATA_KEY_NAME = f"exports/spam-sms/{METADATA_EXPORT_PATH.name}"

In [26]:
session = boto3.session.Session()
client = session.client('s3', region_name=REGION, endpoint_url=ENDPOINT)
client.upload_file(str(MODEL_EXPORT_PATH), BUCKET_NAME,  MODEL_KEY_NAME)
client.upload_file(str(TOKENIZER_EXPORT_PATH), BUCKET_NAME,  TOKENIZER_KEY_NAME)
client.upload_file(str(METADATA_EXPORT_PATH), BUCKET_NAME,  METADATA_KEY_NAME)

In [ ]:
client.download_file(BUCKET_NAME, MODEL_KEY_NAME, pathlib.Path(MODEL_KEY_NAME).name)
client.download_file(BUCKET_NAME, TOKENIZER_KEY_NAME, pathlib.Path(TOKENIZER_KEY_NAME).name)
client.download_file(BUCKET_NAME, METADATA_KEY_NAME, pathlib.Path(METADATA_KEY_NAME).name)

## Implement an AI Model Download Pipeline
In [this blog post](https://www.codingforentrepreneurs.com/blog/ai-model-download-pipeline) I'll show you how to turn the `client.download_file()` portion into a pipeline so you can make it reusable in future projects. Further, if you ever need to bundle these models into a Docker image, you will be able to use the pipeline.